In [ ]:
import pandas as pd
import os
import logging
import numpy as np
import re
from datetime import datetime

# Initialize logging
logging.basicConfig(filename='update_dataframe.log', level=logging.INFO, 
                    format='%(asctime)s %(levelname)s:%(message)s')

# Chemin du dossier contenant les fichiers CSV
folder_path = r"./historique_lit"
# Fichier de sortie Parquet
output_parquet = 'resultat_concatenation_avec_date_heure.parquet'

def extract_datetime_from_filename(file_name):
    """Extrait la date et l'heure à partir du nom de fichier."""
    try:
        # Utiliser regex pour trouver la date (format DD-MM-YYYY) et l'heure (HHMMSS)
        date_match = re.search(r'\d{2}-\d{2}-\d{4}', file_name)
        time_match = re.search(r'\d{6}', file_name)
        
        if date_match and time_match:
            date_part = date_match.group()
            time_part = time_match.group()

            # Conversion des parties date et heure
            date = pd.to_datetime(date_part, format='%d-%m-%Y', errors='coerce')
            formatted_time = f"{time_part[:2]}:{time_part[2:4]}:{time_part[4:6]}"
            datetime_combined = pd.to_datetime(f"{date_part} {formatted_time}", format='%d-%m-%Y %H:%M:%S', errors='coerce')
            return date, formatted_time, datetime_combined
        else:
            return pd.NaT, None, pd.NaT
    except Exception as e:
        logging.error(f"Erreur lors de l'extraction de la date/heure du fichier {file_name}: {e}")
        return pd.NaT, None, pd.NaT

def update_dataframe(folder_path):
    """Met à jour le DataFrame en chargeant tous les fichiers CSV dans le dossier."""
    dataframes = []
    
    # Parcourir tous les fichiers dans le dossier
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            
            try:
                # Extraire la date et l'heure à partir du nom du fichier
                date, formatted_time, datetime_combined = extract_datetime_from_filename(file_name)
                if pd.isna(date) or pd.isna(datetime_combined):
                    logging.warning(f"Le fichier {file_name} a un format de nom invalide.")
                    continue  # Si le format est incorrect, sauter ce fichier

                # Lecture des premières lignes pour vérifier le format
                with open(file_path, 'r', encoding='cp1252', errors='replace') as f:
                    first_line = f.readline()
                    if ";" not in first_line:
                        logging.warning(f"Le fichier {file_name} ne semble pas avoir un bon format.")
                        continue

                # Lecture du fichier CSV
                df = pd.read_csv(file_path, encoding='cp1252', sep=";", on_bad_lines='skip')

                # Vérifier que le DataFrame n'est pas vide ou mal formaté
                if df.empty or df.shape[1] == 1:
                    logging.warning(f"Le fichier {file_name} est vide ou mal formaté.")
                    continue

                # Ajout des colonnes 'Date', 'Heure', et 'DateHeure' extraites du nom du fichier
                df['Date'] = date
                df['Heure'] = formatted_time
                df['DateHeure'] = datetime_combined

                # Création de la colonne 'Localisation'
                df['Localisation'] = df['UNITE'].astype(str) + '-' + df['CHAMBRE'].astype(str).str.strip() + '-' + df['LIT'].astype(str).str.strip()
                
                # Ajouter le DataFrame à la liste
                dataframes.append(df)
            
            except Exception as e:
                logging.error(f"Erreur lors du traitement du fichier {file_name}: {e}")
    
    return dataframes

# Mettre à jour le DataFrame avec tous les fichiers CSV disponibles
new_dataframes = update_dataframe(folder_path)

if new_dataframes:
    # Concaténer avec l'ancien DataFrame si le fichier Parquet existe déjà
    if os.path.exists(output_parquet):
        old_df = pd.read_parquet(output_parquet)
        final_dataframe = pd.concat([old_df] + new_dataframes, ignore_index=True)
    else:
        final_dataframe = pd.concat(new_dataframes, ignore_index=True)

    # Ajouter les colonnes dérivées du champ 'PATIENT'
    final_dataframe['LIT_DISPONIBLE'] = np.where(final_dataframe['PATIENT'] == 'LIT DISPONIBLE', 1, 0)
    final_dataframe['LIT_INDISPONIBLE'] = np.where(final_dataframe['PATIENT'] == 'LIT INDISPONIBLE', 1, 0)
    final_dataframe['EMPLACEMENT_VIDE'] = np.where(final_dataframe['PATIENT'] == 'EMPLACEMENT VIDE', 1, 0)
    final_dataframe['PATIENT_UNKNOWN'] = np.where(final_dataframe['PATIENT'] == '99999999', 1, 0)

    # Sauvegarder le fichier Parquet
    final_dataframe.to_parquet(output_parquet, index=False)
    logging.info(f"Le fichier Parquet a été mis à jour : {output_parquet}")
    
    # Afficher les 5 premières lignes du DataFrame
    print(final_dataframe.head())
    
    # Afficher certaines colonnes
    display(final_dataframe[['PATIENT', 'LIT_DISPONIBLE', 'LIT_INDISPONIBLE', 'EMPLACEMENT_VIDE', 'Date', 'DateHeure']].head())

else:
    logging.info("Aucun nouveau fichier CSV n'a été traité.")
    print("Aucun nouveau fichier CSV n'a été traité.")
